In [2]:

import main
from pars_shocks import Pars, Shocks
import plot_inequality as plot_ineq
import time
import importlib
import io_manager
import numpy as np
importlib.reload(plot_ineq)
#run stuff here
start_time = time.perf_counter()
print("Running main")

# ***** may want to change how trans is generated its redundant in do file.
of_name = None
main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/"
k2_input_path = main_path + "input/k2_moms/"
age_type_input_path = main_path + "input/50p_age_moms/"

trans_full_path = age_type_input_path + "MH_trans_by_MH_clust_age.csv"  
trans_uncond_age_path = age_type_input_path + "MH_trans_uncond_age.csv"

H_type_pop_share_path = age_type_input_path + "MH_clust_50p_age_pop_shares.csv"

# trans_full_path = k2_input_path + "MH_trans_by_MH_clust_age.csv"
# trans_uncond_age_path = k2_input_path + "MH_trans_uncond_age.csv"
# trans_test_path = k2_input_path + "MH_trans_test.csv"

# H_type_pop_share_path = k2_input_path + "MH_clust_k2_pop_shares.csv"

myPars = main.pars_factory(main_path, H_type_pop_share_path = H_type_pop_share_path)
myPars.H_trans = io_manager.read_and_shape_H_trans_full(myPars, path = trans_full_path) 
# myPars.H_trans = io_manager.read_and_shape_H_trans_uncond_age(myPars, trans_uncond_age_path)
myPars, myShocks, sols, sims = main.main_io(main_path, myPars = myPars, out_folder_name = of_name, do_phi_H_calib = False, output_flag = False)


Running main
Using default health transition matrix
*****Running main_io with default out_folder_name*****
Solver ran in 7.308170199998131 seconds
Calibrating with alpha_lab_targ = 0.31842728882352944, w0_mean_targ = 9.4541645, w0_sd_targ = 3.2013602, 
                                        w1_targ = 0.3406973883314617, w2_targ = 0.3031905120658842, wH_targ = 0.035280712, phi_H_target = 0.01319771
***** Calibration iteration 0 *****
***** Calibration iteration 1 *****
***** Calibration iteration 2 *****
***** Calibration iteration 3 *****
***** Calibration iteration 4 *****
***** Calibration iteration 5 *****
***** Calibration iteration 6 *****
***** Calibration iteration 7 *****
***** Calibration iteration 8 *****
***** Calibration iteration 9 *****
***** Calibration iteration 10 *****
***** Calibration iteration 11 *****
***** Calibration iteration 12 *****
Calibration converged after 13 iterations
********** phi_H calibration was skipped **********
w0_weights = [0.28046962 0.593356

In [30]:
import model_uncert as model
importlib.reload(model)
import my_toolbox as tb
importlib.reload(tb)

import io_manager as io
import solver
import simulate

from typing import Tuple, Dict
from numba import njit

def calib_phi_H(myPars: Pars, main_path: str, tol:float, target:float, phi_H_min:float, phi_H_max:float)-> Tuple[float, float, Dict[str, np.ndarray], Dict[str, np.ndarray]]:
    '''
    calibrates the phi_H parameter of the model to match the target difference in mean log hours worked between the bad MH and good MH states
    takes the following arguments:
    myPars: the parameters of the model
    main_path: the path to the main directory
    tol: the tolerance for the calibration
    target: the target difference in mean log hours worked between the bad MH and good MH states
    phi_H_min: the minimum value of the phi_H parameter
    phi_H_max: the maximum value of the phi_H parameter
    returns a tuple with the calibrated phi_H, the difference in mean log hours worked between the bad MH and good MH states, the state solutions and the simulations
    '''
    io.print_params_to_csv(myPars, path = main_path, file_name = "pre_phi_H_calib_params.csv")
    phi_H_moment = -999.999
    state_sols = {}
    sim_lc = {}

    # define the lambda function to find the zero of
    get_phi_H_diff = lambda new_phi_H: phi_H_moment_giv_phi_H(myPars, new_phi_H)[0] - target
    print("Entering bisection search")
    calibrated_phi_H = tb.my_bisection_search(get_phi_H_diff, phi_H_min, phi_H_max, tol, myPars.max_iters, myPars.print_screen)
    myPars.phi_H = calibrated_phi_H

    # solve, simulate model for the calibrated phi_H
    phi_H_moment, state_sols, sim_lc = phi_H_moment_giv_phi_H(myPars, calibrated_phi_H)
    io.print_params_to_csv(myPars, path = main_path, file_name = "phi_H_calib_params.csv")
    if myPars.print_screen >= 1:
        print(f"Calibration exited: phi_H = {calibrated_phi_H}, difference in mean log hours worked = {phi_H_moment}, target difference in mean log hours worked = {target}")

    return calibrated_phi_H, phi_H_moment, state_sols, sim_lc

def phi_H_moment_giv_phi_H(myPars: Pars, new_phi_H:float)-> Tuple[float, Dict[str, np.ndarray], Dict[str, np.ndarray]]:
    '''
    solves the model for a given phi_H and returns the phi_H moment, the mean phi_H, the state solutions and the simulations
    '''
    myPars.phi_H = new_phi_H
    shocks = Shocks(myPars)
    state_sols = solver.solve_lc(myPars)
    sim_lc = simulate.sim_lc(myPars, shocks, state_sols)
    lab_sim_lc = sim_lc['lab']
    return phi_H_moment(myPars, lab_sim_lc, shocks), state_sols, sim_lc

# def phi_H_moment(myPars: Pars, sims: Dict[str, np.ndarray], shocks: Shocks)-> float:
# @njit
def phi_H_moment(myPars: Pars, lab_sims: np.ndarray, shocks: Shocks)-> float:
    '''
    returns the difference in mean log hours worked between the bad MH and good MH states 
    '''
    # lab_sims: np.ndarray = sims['lab'][:, :, :, :myPars.J]
    # print(f"lab_sims mean by age: {np.mean(lab_sims, axis = tuple(range(1, lab_sims.ndim - 1)))}")
    lab_sims = lab_sims[:, :, :, :myPars.J]

    # take the log where hours are positive
    # lab_non_zero_mask = (lab_sims >= 0)
    for i in range(lab_sims.size):
        if lab_sims.flat[i] <= 0:
            lab_sims.flat[i] = 1e-10

    # log_lab_sims = np.log(lab_sims, where = lab_sims > 0)
    # log_lab_sims = np.log(lab_sims * lab_non_zero_mask)

    log_lab_sims = np.log(lab_sims)
    # log_lab_sims = lab_sims
    
    H_hist: np.ndarray = shocks.H_hist[:, :, :, :myPars.J]

    bad_MH_log_lab_sims = log_lab_sims * (H_hist == 0)
    good_MH_log_lab_sims = log_lab_sims * (H_hist == 1)

    bad_MH_log_lab_mean = model.wmean_non_zero(myPars, bad_MH_log_lab_sims)
    good_MH_log_lab_mean = model.wmean_non_zero(myPars, good_MH_log_lab_sims)
    # bad_MH_log_lab_mean = np.mean(bad_MH_log_lab_sims, where = bad_MH_log_lab_sims != 0)
    # good_MH_log_lab_mean = np.mean(good_MH_log_lab_sims, where = good_MH_log_lab_sims != 0)
    diff = good_MH_log_lab_mean - bad_MH_log_lab_mean   

    # print(f"good_MH_log_lab_mean: {good_MH_log_lab_mean} - bad_MH_log_lab_mean: {bad_MH_log_lab_mean} = diff: {diff}")

    return diff
    
def get_phi_H_targ(myPars: Pars, target_folder_path: str)-> float:
    '''
    reads phi_H target moment from myPars.path + '/input/MH_hours_moments.csv'
    '''
    # data_moments_path = myPars.path + '/input/MH_labor_moments.csv'
    data_moments_path = target_folder_path + '/MH_hours_moments.csv'
    data_mom_col_ind = 0
    mean_log_lab_diff = tb.read_specific_column_from_csv(data_moments_path, data_mom_col_ind)
    return mean_log_lab_diff[0]

In [32]:
print_screen = 2
myPars.print_screen = print_screen
phi_H_min = 0.00001
phi_H_max = 0.5
targ_input_path = age_type_input_path 
# phi_H_mom_targ = get_phi_H_targ(myPars, targ_input_path)
phi_H_mom_targ = 0.03
calib_tol = 1e-3

new_wH = np.log(1.0)
myPars.wH_coeff = new_wH

# new_phi_H = 0.1
phi_arr = np.arange(0.001, 0.030, 0.005)
print(f"phi_arr: {phi_arr}")

# for new_phi_H in phi_arr:
#     phi_H_mom = phi_H_moment_giv_phi_H(myPars, new_phi_H)[0]
    # print(f"phi_H_mom: {phi_H_mom}")

# phi_H_mom = phi_H_moment_giv_phi_H(myPars, new_phi_H)[0]
# print(f"phi_H_mom: {phi_H_mom}")

print(f"phi_H_mom_targ: {phi_H_mom_targ}")
calib_phi_H(myPars, main_path, calib_tol, phi_H_mom_targ, phi_H_min, phi_H_max)

phi_arr: [0.001 0.006 0.011 0.016 0.021 0.026]
phi_H_mom_targ: 0.03
Entering bisection search
iteration 0: mid_point = 0.250005, mid_val = 0.6335397245085501
iteration 1: mid_point = 0.1250075, mid_val = 0.23351167285857136
iteration 2: mid_point = 0.06250875, mid_val = 0.08956644610074996
iteration 3: mid_point = 0.031259375, mid_val = 0.022633199057293246
iteration 4: mid_point = 0.0156346875, mid_val = -0.009805060969839213
iteration 5: mid_point = 0.02344703125, mid_val = 0.006334080913188461
iteration 6: mid_point = 0.019540859375, mid_val = -0.001755086923830057
iteration 7: mid_point = 0.0214939453125, mid_val = 0.0022845484727205545
iteration 8: mid_point = 0.020517402343749998, mid_val = 0.00026349984985810093
Calibration exited: phi_H = 0.020517402343749998, difference in mean log hours worked = 0.0302634998498581, target difference in mean log hours worked = 0.03


(0.020517402343749998,
 0.0302634998498581,
 {'c': array([[[[[8.79857812e-01, 8.79506205e-01, 8.77786525e-01, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04],
            [9.03459041e-01, 9.03257558e-01, 9.01430690e-01, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04]],
  
           [[8.81082873e-01, 8.80815211e-01, 8.79091023e-01, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04],
            [9.04558106e-01, 9.04281082e-01, 9.02655440e-01, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04]]],
  
  
          [[[2.93948887e+00, 2.95246029e+00, 2.96329978e+00, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04],
            [2.98666288e+00, 2.99993353e+00, 3.01055759e+00, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04]],
  
           [[2.94193900e+00, 2.95507836e+00, 2.96590879e+00, ...,
             1.00000000e-04, 1.00000000e-04, 1.00000000e-04],
            [2.98886091e+00, 3.00198061